<h1>Groupe 1 - Data Collection
> Rome2Rio scrap script
<span class="tocSkip"></span>
> *Authors : All*

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Loading</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions</a></span></li></ul></li><li><span><a href="#Crawl" data-toc-modified-id="Crawl-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Crawl</a></span></li><li><span><a href="#Descriptive-statistics-on-recovered-data" data-toc-modified-id="Descriptive-statistics-on-recovered-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Descriptive statistics on recovered data</a></span></li></ul></div>

# Introduction

One of the sources we need to scrap was the Rome2Rio website. This notebook contains the code to retrieve the data with Chromedriver driver. To have the new data of Rome2Rio, we're going to launch manually this script every month. We scrapped the itinerary between all European capitals. They contain the departure and arrival city, the number of transports taken, the price minimum and maximum and also the duration of the itinerary.

V0 : basic scrap 
We retrieve all the informations of the website and we put the results into a json.

V1 : Quality in the code 
Begin of respect of quality chart

V2 : Generalization 
Creation of statistics for Rome2Rio. We finally arrived to scrapp 13 674 row.

V_not used : We try to adapt the code with the chromedriver to phantomjs but that don't work.

# Environment

## Libraries

In [2]:
import ast
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import re

## Data Loading

## Functions

This part contains all the functions we developped during the project for Rome2rio

In [3]:
def getdata(departure_city, arrival_city):
    """Documentation

    Parameters:

        departure_city: The departure city
        arrival_city: The arrival city

    Return:

        datafinal: The final dataframe with the variables imposed in the data dictionary for rom2rio

    """
    # Activation of Chrome Options

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    webdriver = "../Driver/chromedriver.exe"

    driver = Chrome(webdriver, chrome_options=chrome_options)
    # The link of the website we want to scrap
    url = "https://www.rome2rio.com/map/"+departure_city+"/"+arrival_city
    driver.get(url)

    # The number of items we have by class
    items = len(driver.find_elements_by_class_name("route__title"))
    items2 = len(driver.find_elements_by_class_name("route__details"))

    total = []
    data = pd.DataFrame()

    # driver.find_elements_by_class_name : to locate an element by class attribute name
    for i in range(items):
        a = driver.find_elements_by_class_name("route__title")[i].text
        b = driver.find_elements_by_class_name("route__details")[i].text

        # nb days
        jour = "days"
        if jour in a:
            nb_days = int(
                ((re.findall("[0-9]+"+" days", a))[0]).replace(jour, ""))
        else:
            nb_days = 0
        # nb hours
            nb_hours = (re.search("[0-9]+"+"h", a))
            if nb_hours:
                nb_hours = int(
                    ((re.search("[0-9]+"+"h", a)).group(0)).replace("h", ""))
            else:
                nb_hours = 0

        # nb minute
        nb_minute = re.search("[0-9]+"+"m", a)
        if nb_minute:
            nb_minute = int((nb_minute.group(0)).replace("m", ""))
        else:
            nb_minute = 0

        new = ((a, b, nb_days, nb_hours, nb_minute))
        total.append(new)

    data = pd.DataFrame(total, columns=[
                        'itineraire', 'prix', 'liste_jour', 'liste_hours', 'liste_minute'])

    data['Data_Source'] = 'Rome2rio'
    data['Departure_city'] = departure_city
    data['Arrival_city'] = arrival_city
    # Count which transport we take and how many times
    data['Nb_bus_taken'] = data.itineraire.str.count("Bus|bus")
    data['Nb_train_taken'] = data.itineraire.str.count("Train|train")
    data['Nb_car_taken'] = data.itineraire.str.count(
        "Drive|drive|Rideshare|rideshare|car|Car")
    data['Nb_plane_taken'] = data.itineraire.str.count("Fly|fly")

    # Give the minimal price and the maximum price
    data['Price_max'] = data['prix'].apply(lambda st: st[st.find("-")+1:])
    data['Price_min'] = data['prix'].apply(lambda st: st[:st.find("-")])
    data['Duration1'] = data['itineraire'].apply(
        lambda st: st[st.find("•")+1:])

    data['days'] = data['liste_jour']
    data['hours'] = data['liste_hours']
    data['minute'] = data['liste_minute']

    data['days'] = data['days'].replace('', '0')
    data['minute'] = data['minute'].replace('', '0')
    data['hours'] = data['hours'].replace('', '0')

    data['Duration'] = data['days']*24*60+data['hours']*60+data['minute']

    datafinal = data[['Data_Source', 'Departure_city', 'Arrival_city', 'Nb_bus_taken',
                      'Nb_train_taken', 'Nb_car_taken', 'Nb_plane_taken', 'Duration', 'Price_min', 'Price_max']]

    return datafinal

# Crawl

This part contains the crawl of Rome2rio website with the execution of all functions 

To load the script on all European capitals, we can use this list below and replace it instead of example_capitale

liste_capitale = ["Paris",
                  "Berlin",
                  "Rome",
                  "Madrid",
                  "London",
                  "Dublin",
                  "Lisbon",
                  "Brussels",
                  "Luxembourg",
                  "Amsterdam",
                  "Bern",
                  "Copenhagen",
                  "Oslo",
                  "Stockholm",
                  "Helsinki",
                  "Tallinn",
                  "Riga",
                  "Vilnius",
                  "Warsaw",
                  "Prague",
                  "Vienna",
                  "Bratislava",
                  "Budapest",
                  "Ljubljana",
                  "Ankara",
                  "Bucharest",
                  "Belgrade",
                  "Sofia",
                  "Tijana",
                  "Skopje",
                  "Athens",
                  "Chisinau",
                  "Kiev",
                  "Minsk",
                  "Moscow",
                  "Tbilissi",
                  "Bakou",
                  "Verevan",
                  "Sarajevo",
                  "Reykjavik",
                  "Valletta",
                  "Zagreb",
                  "Nicosia",
                  "Andorra la Vella",
                  "San Marino",
                  "Vatican City"]

In [1]:
liste_capitale = ["Paris", "Berlin", "Rome", "Madrid", "London", "Dublin", "Lisbon", "Brussels", "Luxembourg", "Amsterdam",
                  "Bern", "Copenhagen", "Oslo", "Stockholm", "Helsinki", "Tallinn", "Riga", "Vilnius", "Warsaw", "Prague", "Vienna", "Bratislava",
                  "Budapest", "Ljubljana", "Ankara", "Bucharest", "Belgrade", "Sofia", "Tijana", "Skopje", "Athens", "Chisinau", "Kiev", "Minsk",
                  "Moscow", "Tbilissi", "Bakou", "Verevan", "Sarajevo", "Reykjavik", "Valletta", "Zagreb", "Nicosia", "Andorra la Vella",
                  "San Marino", "Vatican City"]

In [5]:
# To run the function getdata to a list of capitale
tmp = [[x, y] for x in liste_capitale for y in liste_capitale if x != y]
appended_data = []
for i in range(len(tmp)):
    appended_data.append(getdata(tmp[i][0], tmp[i][1]))
appended_data = pd.concat(appended_data, ignore_index=True)

/home/sid2018-5/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: use options instead of chrome_options


In [6]:
appended_data.head(3)

,Data_Source,Departure_city,Arrival_city,Nb_bus_taken,Nb_train_taken,Nb_car_taken,Nb_plane_taken,Duration,Price_min,Price_max
0,Rome2rio,Paris,Berlin,0,0,0,1,285,47€,224€
1,Rome2rio,Paris,Berlin,0,0,0,1,281,49€,277€
2,Rome2rio,Paris,Berlin,0,0,0,1,293,57€,326€


In [7]:
# The others we want to have in the final dataframe
col_names = ['Date_Review', 'Review', 'Airline_Name', 'Airline_Type', 'Region_Operation', 'Aircraft_Type', 'Cabin_Class', 'Type_Of_Lounge',
             'Type_Of_Traveller', 'Date_Visit', 'Date_Flown', 'Airport', 'Route', 'Category', 'Category_Detail',
             'Cabin_Staff_Service', 'Lounge_Staff_Service', 'Bar_And_Beverages', 'Food_And_Beverages', 'Ground_Service', 'Catering', 'Cleanliness',
             'Lounge_Comfort', 'Aisle_Space', 'Wifi_And_Connectivity', 'Inflight_Entertainment', 'Viewing_Tv_Screen', 'Power_Supply',
             'Seat', 'Seat_type', 'Seat_Comfort', 'Seat_Legroom', 'Seat_Storage', 'Seat_Width', 'Seat_Recline', 'Washrooms',
             'Value_For_Money', 'Overall_Customer_Rating', 'Overall_Service_Rating', 'Overall_Airline_Rating',
             'Recommended', 'Title', 'Author', 'Description', 'Date_publication',
             'View_count', 'Likes', 'Dislikes', 'Nb_subscribers', 'Nb_comments', 'Nb_sharing', 'Hashtags', 'Awards', 'Registration', 'Location',
             'Contributions_Pers', 'Nb_pertinent_comments_Pers', 'Queuing_Times', 'Terminal_Seating', 'Terminal_Signs', 'Airport_Shopping',
             'Experience_At_Airport']
datafinal = pd.DataFrame(columns=col_names)

In [8]:
# Concatenate to have all features
rom2rio_ = pd.concat([appended_data, datafinal], sort=False)

In [9]:
# Export the final result to json
rom2rio = rom2rio_.to_json()
with open('../Results_json/data_Rome2Rio.json', 'a', encoding='utf8') as outfile:
    json.dump(rom2rio, outfile, ensure_ascii=False, indent=4)

# Descriptive statistics on recovered data

In [19]:
# Load the result of all european city
r2r = pd.read_csv("../Results_json/data_Rome2Rio.csv", sep=";")

In [21]:
r2r[['Departure_city', 'Arrival_city', 'Duration']].groupby(['Departure_city', 'Arrival_city']).agg(
    [('Min', 'min'), ('Mean', 'mean'), ('Median', 'median'), ('Max', 'max'), ('Count', 'count')])

Duration                                 
                                     Min         Mean  Median   Max Count
Departure_city Arrival_city                                              
Amsterdam      Andorra la Vella      478   944.333333   875.0  1902     9
               Athens                390  1681.818182  1931.0  3000    11
               Bakou                 636  3008.272727  3180.0  6300    11
               Belgrade              319  1005.700000  1093.5  1840    10
               Berlin                240   358.888889   354.0   585     9
               Bern                  301   442.333333   427.0   763     9
               Bratislava            295   645.200000   537.5  1142    10
               Brussels              113   160.333333   147.5   262     6
               Bucharest             366  1275.100000  1515.5  2246    10
               Budapest              299   809.200000   825.0  1554    10
               Chisinau              460  1678.857143  1855.0  2798     7
               Copenhagen            245   562.714286   513.0   917     7
               Dublin                259   667.333333   563.5  1476     6
               Helsinki              313  1374.875000  1167.5  2639     8
               Kiev                  378  1130.875000   884.5  2143     8
               Lisbon                356  1064.777778  1260.0  1884     9
               Ljubljana               0   712.363636   679.0  1319    11
               London                213   305.000000   263.5   660    12
               Luxembourg            162   353.400000   370.0   627     5
               Madrid                332   843.333333   867.0  1665     9
               Minsk                 357  1233.666667  1252.0  2217     6
               Moscow                388  1197.200000   521.0  3000    10
               Nicosia               470  1156.375000   570.5  3480     8
               Oslo                  279  1002.700000  1076.5  1741    10
               Paris                 203   298.700000   271.0   445    10
               Prague                254   494.666667   378.0   780     9
               Reykjavik             415   504.333333   534.0   564     3
               Riga                  348  1100.875000   796.0  2104     8
               Rome                  310   801.333333   665.0  1665    12
               San Marino            432   857.727273   808.0  1340    11
...                                  ...          ...     ...   ...   ...
Zagreb         Helsinki              205  2279.166667  2543.5  3300     6
               Kiev                  339  1071.666667  1265.5  1678     6
               Lisbon                257  2112.000000  2367.0  3000     7
               Ljubljana              87   122.333333   128.0   145     6
               London                194  1042.222222  1006.0  2050     9
               Luxembourg            243   891.500000  1019.0  1182     8
               Madrid                238  1794.666667  1607.0  3780     6
               Minsk                 328  1364.200000  1642.0  2170     5
               Moscow                274  1451.142857  1503.0  2861     7
               Nicosia               401   898.000000   564.0  1729     3
               Oslo                  210  1524.166667  1720.0  2201     6
               Paris                 187   990.500000  1050.0  1495     6
               Prague                130   514.285714   595.0   733     7
               Reykjavik             517   517.000000   517.0   517     1
               Riga                  374  1620.500000  1923.5  2245     6
               Rome                  266   588.600000   657.0   780     5
               San Marino            368   590.444444   595.0   809     9
               Sarajevo               74   351.571429   375.0   517     7
               Skopje                189   436.500000   467.5   622     4
               Sofia                 268   575.000000   608.5   815     4
               Stockholm            